# 自动求导 代码实现

In [1]:
import torch

## 1 y 是标量

In [2]:
x = torch.arange(4.0) # x 是列向量
x

tensor([0., 1., 2., 3.])

求导： y = 2 [X.T] [X]，显然y是标量，x是向量

In [3]:
# 在反向传播之前，我们需要有地方来存储梯度
x.requires_grad_(True) # 开启自动微分
x.grad # 默认None，以后可以通过这个属性访问x的梯度

In [4]:
y = 2 * torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

`grad_fn=<MulBackward0>`这个是y的属性，表示y是在前向传播（x->y）中通过乘法操作生成的，这个属性会在反向传播时帮助 PyTorch 确定如何计算梯度

In [5]:
# 调用反向传播函数计算y关于x的各个分量的梯度，结果存入x.grad中
y.backward()
x.grad, y

(tensor([ 0.,  4.,  8., 12.]), tensor(28., grad_fn=<MulBackward0>))

In [6]:
# 标答应该就是4，y = 2 X.T X 关于X求导应该是 4X
x.grad == 4 * x

tensor([True, True, True, True])

In [11]:
# 在默认情况下，pytorch 会将梯度累计，我们如要计算另外的梯度，需要手动清零
x.grad.zero_()
y = x.sum() # y = x1 + x2 +...
y

tensor(6., grad_fn=<SumBackward0>)

In [12]:
y.backward()
x.grad

tensor([1., 1., 1., 1.])

再来看看梯度是怎么累计的：

In [13]:
# 重复一下：
y.backward()
x.grad

tensor([2., 2., 2., 2.])

pytorch会默认累计梯度，这是他们设计上的理念，是为了处理大批量的梯度时能够分步计算而累加

## 2 y 是向量

理论上，向量y关于向量x的导数应该是一个矩阵，但是在深度学习中，我们很少会这么做。

大部分情况我们都是对一个标量来求导（loss一般是标量，向量loss比较麻烦），所以y是向量时，一般先求和再求导

In [15]:
x.grad.zero_()
y = x * x
y,x

(tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>),
 tensor([0., 1., 2., 3.], requires_grad=True))

In [16]:
y.sum().backward()  # y = x1 ^2 + x2 ^2 + ...
# 等价于y.backward(torch.ones(len(x))), 即y和一个全1向量作点积后再求导
x.grad

tensor([0., 2., 4., 6.])

## 3 分离计算

即： y = f(x), z = g(y,x) 我们想求z对x的梯度

思路：把y看成常数，而不是x的函数

In [19]:
x.grad.zero_()
y = x * x
x,y

(tensor([0., 1., 2., 3.], requires_grad=True),
 tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>))

In [20]:
u = y.detach()  # u = [0,1,4,9]，但是u不在x,y的计算图里，是个常数
z = u * x
u, z

(tensor([0., 1., 4., 9.]),
 tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>))

In [21]:
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

所以，z对x求导，把u当成了常数。而y还是x的函数，所以y还可以对x求导

In [22]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## 4 Python控制流的梯度计算

隐式构造的优势

PyTorch 的自动微分功能如何处理带有条件分支和循环的代码:

In [23]:
# 手动构造一个控制流：
def f(a):
    b = a * 2
    while b.norm() < 1000:  # b的每个元素的平方和再开根号
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [24]:
a = torch.randn(size=(), requires_grad=True) # 一个随机标量
a

tensor(-1.1571, requires_grad=True)

In [25]:
d = f(a)
d.backward()
a.grad == d / a

tensor(True)

- 隐式构造：当定义并执行 f(a) 这个函数时，PyTorch 并不预先知道整个计算图的形状和大小。相反，它随着代码的执行，**逐步**构建计算图。
    - 例如，在 while b.norm() < 1000: 这个循环中，PyTorch会根据 b 的当前值决定是否继续执行循环，并且每次迭代时都会动态地添加相应的操作到计算图中。

所以，我们有如下结论：在 PyTorch 中，计算图是动态构建的，这意味着计算图可以在代码执行时，根据控制流（如 `if` 语句、`while` 循环）动态生成。PyTorch 能够追踪这些操作并自动计算梯度。这种特性使得 PyTorch 在处理复杂的控制流时非常灵活和强大。

流程总结：a开启了自动微分，然后进入f函数，此时，对a做的所有操作都是前向传播，而且被记录到计算图中了，计算图也在此时被构建，所有的结果存入d。然后d.backward()会基于前向的路径反向走，对a求导。